In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'transformers' already exists and is not an empty directory.
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [57]:
# model_name_or_path = 'google/mt5-small'
# model_name_or_path = '/content/drive/MyDrive/mt5-pquad/checkpoint-11764'
model_name_or_path = '/content/drive/MyDrive/mt5-pquad/checkpoint-3790'
from transformers import AutoTokenizer, MT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = MT5ForConditionalGeneration.from_pretrained(model_name_or_path)

In [58]:
# from datasets import concatenate_datasets, load_dataset

# data_files = {"train": "/content/drive/MyDrive/train.json", "validation": "/content/drive/MyDrive/dev.json"}
# dataset = load_dataset("json", data_files=data_files)

# train_data = dataset['train']
# valid_data = dataset['validation']

# aug_data = load_dataset("json", data_files='/content/drive/MyDrive/total_augmented.json')


# concat_dataset = concatenate_datasets([train_data, valid_data])
# concat_dataset = concatenate_datasets([concat_dataset, aug_data])
# dataset=concat_dataset

In [59]:
from datasets import Dataset
import json

data_files = ['total_augmented.json', 'train.json', 'dev.json']

total_data_list = []

for file in data_files:
  path = '/content/drive/MyDrive/' + file
  with open(path, 'r') as f:
    content = json.load(f)
    total_data_list = total_data_list + content


dataset = Dataset.from_list(total_data_list)


In [60]:
dataset

Dataset({
    features: ['context', 'pq_id', 'question', 'translation_file', 'answers'],
    num_rows: 8520
})

In [61]:
small_dataset = dataset

# small_dataset= dataset.filter(lambda example: len(example['answers'][0]['text'])>0)
# small_dataset= small_dataset.filter(lambda example: len(example['context'])<600)
# small_dataset= small_dataset.filter(lambda example: 'چرا' not in example['context'])

split_data = small_dataset.train_test_split(test_size=0.1, seed= 42, shuffle=True)

dataset = split_data

del split_data

In [62]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'pq_id', 'question', 'translation_file', 'answers'],
        num_rows: 7668
    })
    test: Dataset({
        features: ['context', 'pq_id', 'question', 'translation_file', 'answers'],
        num_rows: 852
    })
})

In [63]:
def format_dataset(example):
  return {'input': 'answer: ' + example['answers'][0]['text'] + ' context: ' + example['context'], 'target': example['question']}


dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/7668 [00:00<?, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

In [64]:
test_data = dataset['test']

test_data[0]

{'input': 'answer: کسی که به سراغ تو می\u200cآید و کوشش می\u200cکند، و از خدا ترسان است context: از اینکه نابینایی به سراغ او آمده بود! تو چه می\u200cدانی شاید او پاکی و تقوا پیشه کند، یا متذکّر گردد و این تذکّر به حال او مفید باشد! امّا آن کس که توانگر است، تو به او روی می\u200cآوری، در حالی که اگر او خود را پاک نسازد، چیزی بر تو نیست! امّا کسی که به سراغ تو می\u200cآید و کوشش می\u200cکند، و از خدا ترسان است، تو از او غافل می\u200cشوی! هرگز چنین نیست که آنها می\u200cپندارند؛ این  تذکّر و یادآوری است،',
 'target': 'خداوند سفارش فرموده است که از چه کسی نباید غافل شد؟'}

In [65]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings


In [66]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

dataset.set_format(type='torch', columns=columns)

Map:   0%|          | 0/7668 [00:00<?, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

In [67]:
# from transformers import default_data_collator
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [68]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mt5-pquad",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    save_strategy="epoch",
    save_steps=8000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    # push_to_hub=False
    #fp16=True,
)

In [69]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [70]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [71]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.636600,0.516716


TrainOutput(global_step=1917, training_loss=0.6586978695451063, metrics={'train_runtime': 747.2747, 'train_samples_per_second': 10.261, 'train_steps_per_second': 2.565, 'total_flos': 4466232974868480.0, 'train_loss': 0.6586978695451063, 'epoch': 1.0})

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prompt = "answer: طور context: اي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
#                                     top_k=10,
#                                     top_p=.2,
#                                     temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)

بنی اسرائيل در طرف راست چه چیز با شما وعده گذاردند؟
